# Task 1 - Simple Linear Regression

- a) Perform a simple linear regression on the `Auto.csv` dataset. Use `mpg` as the response variable and `horsepower` as the predictor variable.
    - i) Is there a relationship between the predictor and the response?
    - ii) How strong is the relationship between the predictor and the response?
    - iii) Is the relationship between the predictor and the response positive or negative?
    - iv) What is the predicted `mpg` associated with a `horsepower` of 98? What are the associated 95% confidence and prediction intervals?
- b) Plot the response and the predictor. Display the least squares regression line.
- c) Perform a multiple linear regression on the dataset. Use `mpg` as the response variable and both `horsepower` and `weight` as the predictor variables. Use `statsmodels` to solve this task (instead of doing it manually). Does this model perform better than the simple linear regression?

In [1]:
import pandas as pd

auto_df = pd.read_csv('Auto.csv')

auto_df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
393,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
394,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
395,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [2]:
auto_df['horsepower'] = pd.to_numeric(auto_df['horsepower'], errors='coerce')
auto_df = auto_df.dropna(subset=['horsepower'])

In [3]:
import plotly.graph_objects as go
import statsmodels.api as sm

x = auto_df['horsepower']
y = auto_df['mpg']

X = sm.add_constant(x)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Data'))

fig.add_trace(go.Scatter(x=x, y=predictions, mode='lines', name='Regression Line'))

fig.update_layout(
    title="Regression Plot of Horsepower vs MPG",
    xaxis_title="Horsepower",
    yaxis_title="MPG"
)

fig.show()

ModuleNotFoundError: No module named 'statsmodels'

i) Is there a relationship between the predictor and the response?

In [ ]:
# let's check with the null hypothesis that there is no relationship between the predictor and the response
import statsmodels.api as sm

X = auto_df['horsepower']
X = sm.add_constant(X)
y = auto_df['mpg']
model = sm.OLS(y, X).fit()
model.summary()


We could stop here, we have what we need to answer the question. Let's go over this table step-by-step, because it looks overwhelming at first.

For question i) the rows `const` and `horsepower` are interesting:

```
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
```


In our case, what we are doing with our linear regression is to estimate the following equation:

$$
\text{mpg} = \beta_0 + \beta_1 \times \text{horsepower} + \epsilon
$$

That is to say, we are estimating a simplified version of the function of `mpg`, based on `horsepower`. We include the error term $\epsilon$ because we know that our model will not be perfect - there could be some fluctuations in `mpg` that we cannot explain with `horsepower` alone.
Possible reasons for this include (this list is not exhaustive!):
- The tools used to measure `mpg` are not perfect (all tools have some margin of error)
- There are other variables that influence `mpg` that we are not considering (such as the weight of the car - which is almost definitely a factor)
- Some random fluctuations that we cannot explain (but this could get philosophical - maybe our physical understanding of the universe is incomplete and there are some factors that we cannot explain yet, so even with perfect tools and all variables considered, there might still be fluctuations)

In our table above, `const` is $\hat{\beta_0}$ and `horsepower` is $\hat{\beta_1}$. `statsmodels` has solved the linear regression for us and found the best values for $\hat{\beta_0}$ and $\hat{\beta_1}$ (which estimate the actual $\beta_0$ and $\beta_1$). The values are:
- $\hat{\beta_0} = 39.9359$
- $\hat{\beta_1} = -0.1578$



Let's calculate these values by hand!

To estimate the coefficients:

$$
\begin{flalign*}
\hat{\beta_0} &= \bar{y} - \hat{\beta_1} \times \bar{x} \\
\hat{\beta_1} &= \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n}(x_i - \bar{x})^2}
\end{flalign*}
$$


Let's calculate $\bar{x}$ first.

$$\bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

In [ ]:
mean_x = 1 / len(X) * sum(X['horsepower'])
mean_x

Let's calculate $\hat{\beta_1}$ next. This is the slope of the regression line. It tells us how much `mpg` changes for a one-unit increase in `horsepower`.

In [ ]:
beta1 = sum((X['horsepower'] - mean_x) * (y - y.mean())) / sum((X['horsepower'] - mean_x) ** 2)
beta1

And now $\hat{\beta_0}$ - this is the intercept of the regression line.

In [ ]:
beta0 = y.mean() - beta1 * mean_x
beta0

Next, let's calculate $\sigma$.

**Note**: This is *not* the same as the standard deviation. What we need here is the standard error of the residuals. The formula is:

$$\sigma = \sqrt{\frac{\Sigma_{i=1}^n(y_i - \hat{y}_i)^2}{n-2}} = \sqrt{\frac{\Sigma_{i=1}^n \left( y_i - \left( \hat{\beta_0} + \hat{\beta_1} * x_i \right) \right)^2 }{n - 2}}$$

You can find this in the lecture slides as well. It says there:

$$\sigma^2 = Var(\epsilon)$$

From this it follows that $\sigma = \sqrt{Var(\epsilon)}$.

Why do we have to do divide by $n-2$? This is because we have estimated two parameters: $\beta_0$ and $\beta_1$. This is called the degrees of freedom. The calculation of our two parameters $\beta_0$ and $\beta_1$ has used up two degrees of freedom. So we have $n-2$ degrees of freedom left for the residuals. 

 Let's look at an example, we have a population of exactly 100 points, and we want to calculate the variance of this population. But we only have a sample size of 3 points! We can calculate the variance of this sample, but it will not be the same as the variance of the population. But if we repeat this process many times, and take the average of the sample variances, we will approach the true variance of the population.

In [ ]:
import numpy as np

np.random.seed(42)
population = np.random.randint(0, 100, 100)

true_variance = sum((population - population.mean()) ** 2) / 100

# take a sample of 3 points
sample = np.random.choice(population, 3)
sample_mean = np.mean(sample)
sample_variance = sum((sample - sample_mean) ** 2) / (3 - 1)

true_variance, sample_variance

We see that this is not close. But that is because we have a small sample size. If we repeat this process, and take the average over many sample variances, we will approach the true variance.

In this example below the naive way to calculate the variance would be to divide by the sample size. But you need to remember that the sample mean is not the true mean, but rather already an estimate. That means our sample mean is "drawn" towards the sample points. The deviations from the sample mean are therefore, **on average**, smaller than the deviations from the true mean. Once we know the sample mean, one of our points becomes fixed, it can not vary anymore. That means we have lost one degree of freedom. That is why we divide by $n-1$ and not by $n$.

In [ ]:
sample_variances = []
sample_variances_wrong = []
for _ in range(10_000):
    sample = np.random.choice(population, 3)
    sample_mean = np.mean(sample)
    sample_variance = sum((sample - sample_mean) ** 2) / (3 - 1)
    sample_variances.append(sample_variance)

    # we do not divide by the degrees of freedom
    sample_variance_wrong = sum((sample - sample_mean) ** 2) / 3
    sample_variances_wrong.append(sample_variance_wrong)

In [ ]:
cumulative_mean_diffs = []
cumulative_mean_variance = 0

cumulative_mean_diffs_wrong = []
cumulative_mean_variance_wrong = 0
for i in range(len(sample_variances)):
    cumulative_mean_variance = np.mean(sample_variances[:i + 1])
    difference = cumulative_mean_variance - true_variance
    cumulative_mean_diffs.append(difference)

    cumulative_mean_variance_wrong = np.mean(sample_variances_wrong[:i + 1])
    difference_wrong = cumulative_mean_variance_wrong - true_variance
    cumulative_mean_diffs_wrong.append(difference_wrong)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(cumulative_mean_diffs))),
    y=cumulative_mean_diffs,
    mode='lines',
    name='Cumulative Mean Difference'
))

fig.add_trace(go.Scatter(
    x=list(range(len(cumulative_mean_diffs_wrong))),
    y=cumulative_mean_diffs_wrong,
    mode='lines',
    name='Cumulative Mean Difference (Wrong)'
))

fig.update_layout(
    title='Convergence of Sample Variance to True Variance',
    xaxis_title='Sample Number',
    yaxis_title='Difference from True Variance',
    yaxis=dict(tickformat=".4f"),
    showlegend=True
)

fig.show()

The same principle applies to the residual standard error. We have estimated two parameters ($\hat{\beta_0}$ and $\hat{\beta_1}$), so need to compensate for that in the denominator.

In [ ]:
from math import sqrt

residual_std_error = sqrt(sum((y - (beta0 + beta1 * X['horsepower'])) ** 2) / (len(X) - 2))
residual_std_error

Let's also calculate the confidence intervals.

From the lecture we know that:

$$
\begin{flalign*}
SE(\hat{\beta_0})^2 &= \sigma^2 \left[ \frac{1}{n} + \frac{\bar{x}^2}{\sum_{i=1}^{n}(x_i - \bar{x})^2} \right] \\
SE(\hat{\beta_1})^2 &= \frac{\sigma^2}{\sum_{i=1}^{n}(x_i - \bar{x})^2}
\end{flalign*}
$$


Next, the standard error of $\hat{\beta_0}$.

In [ ]:
se_beta0 = residual_std_error * sqrt(1 / len(X) + mean_x ** 2 / sum((X['horsepower'] - mean_x) ** 2))
se_beta0

And the standard error of $\hat{\beta_1}$.

In [ ]:
se_beta1 = residual_std_error / sqrt(sum((X['horsepower'] - mean_x) ** 2))
se_beta1

Until here, we have calculated the following values:

- $\hat{\beta_0} = 39.9359$
- $\hat{\beta_1} = -0.1578$
- $\sigma = 4.90575691954594$
- $SE(\hat{\beta_0}) = 0.7174986555545253$
- $SE(\hat{\beta_1}) = 0.006445500517685023$

In [ ]:
beta0, beta1, residual_std_error, se_beta0, se_beta1

```
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
```

Now we can calculate the confidence intervals for $\hat{\beta_0}$ and $\hat{\beta_1}$.

The formula for the confidence interval of $\hat{\beta_0}$ is:

$$
\hat{\beta_0} \pm 2 \times SE(\hat{\beta_0})
$$

And for $\hat{\beta_1}$:

$$
\hat{\beta_1} \pm 2 \times SE(\hat{\beta_1})
$$



In [ ]:
ci_beta0 = (beta0 - 2 * se_beta0, beta0 + 2 * se_beta0)
ci_beta0

In [ ]:
ci_beta1 = (beta1 - 2 * se_beta1, beta1 + 2 * se_beta1)
ci_beta1

These values are very slightly different from the ones that `statsmodels` gives us (look at the CI for $\beta_0$). This is because the value $2$ we used in the calculation of the confidence intervals is only an approximation. To replicate the values from `statsmodels` exactly, we need to use the t-distribution.

In [ ]:
from scipy.stats import t

degrees_of_freedom = len(X) - 2
t_value = t.ppf(0.975, degrees_of_freedom)
t_value

Let's plot the t-distribution for our case.

In [ ]:
x = np.linspace(-4, 4, 1000)
y_t_dist = t.pdf(x, degrees_of_freedom)

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y_t_dist, mode='lines', name='t-distribution'))

fig.add_trace(go.Scatter(x=x, y=y_t_dist, fill='tozeroy',
                         fillcolor='rgba(0,100,80,0.2)',
                         line=dict(color='rgba(255,255,255,0)'),
                         showlegend=False,
                         hoverinfo="skip"))

fig.add_trace(go.Scatter(x=[t_value, t_value], y=[0, max(y_t_dist)], mode='lines',
                         line=dict(color='red', dash='dash'),
                         name=f'Critical t-value = {t_value:.2f}'))

fig.update_layout(title=f'T-Distribution with {int(degrees_of_freedom)} Degrees of Freedom',
                  xaxis_title='t-value',
                  yaxis_title='Probability Density',
                  showlegend=True)

fig.show()

The critical t-value tells us where 97.5% of our data lies. We use this for hypothesis testing, to see how likely it is that our data lies within a certain range. If we fall outside of this range (on the right side), then this is very unusual - because 97.5% of our data lies left of the critical t-value.

In [ ]:
ci_beta0 = (beta0 - t_value * se_beta0, beta0 + t_value * se_beta0)
ci_beta0

In [ ]:
ci_beta1 = (beta1 - t_value * se_beta1, beta1 + t_value * se_beta1)
ci_beta1

Let's compare them to the values from `statsmodels`:

```
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
```

Next is the `t` value in the `statsmodels` output. This is **not** the critical t-value, but rather the *t-statistic*. The t-statistic is calculated as:

$$
t = \frac{\hat{\beta_1} - \beta_1}{SE(\hat{\beta})}
$$

Now think all the way back to the beginning - what are we trying to do here?

We want to test the null hypothesis that there is no relationship between `horsepower` and `mpg`. In other words, we want to test if $\beta_1 = 0$. If this is the case, then our estimation for `mpg` becomes:

$$
\text{mpg} = \beta_0 + 0 * \beta_1 + \epsilon = \beta_0 + \epsilon
$$

So to calculate the t-statistic for $\beta_1$, we use the formula:

$$
t = \frac{\hat{\beta_1} - 0}{SE(\hat{\beta_1})}
$$

If it turns out that there **is** a relationship between `horsepower` and `mpg`, then we will reject the null hypothesis and accept the alternative hypothesis that $\beta_1 \neq 0$.

In [ ]:
t_statistic_beta1 = beta1 / se_beta1
t_statistic_beta1

In [ ]:
t_statistic_beta0 = beta0 / se_beta0
t_statistic_beta0

If the magnitude of the t-statistic is larger than the critical t-value, then we can reject the null hypothesis. This means that there is a relationship between the coefficient and `mpg`. This is used to calculate the p-value, which is the probability of observing a t-statistic as extreme as the one we have calculated, given that the null hypothesis is true.

In [ ]:
p_value_beta1 = 2 * (1 - t.cdf(np.abs(t_statistic_beta1), degrees_of_freedom))
p_value_beta1

In [ ]:
p_value_beta0 = 2 * (1 - t.cdf(np.abs(t_statistic_beta0), degrees_of_freedom))
p_value_beta0

In this case we have $P>|t| = 0.000$ for both $\beta_0$ and $\beta_1$. This means that we can reject the null hypothesis that there is no relationship between the predictor variables (`horsepower` and the intercept) and `mpg`, as captured by $\beta_0$ and $\beta_1$. This leads us to accepting the alternative hypothesis that there is a relationship between the predictor variables and `mpg`.

ii) How strong is the relationship between the predictor and the response?

In the lecture we have seen the Residual Standard Error. It is the average deviation of response values from the true regression line. Even if we knew $\beta_0$ and $\beta_1$ we would not be able to perfectly predict $Y$ from $X$.

The formula for the RSE is:

$$
RSE = \sqrt{\frac{1}{n-2} RSS}
$$

And the RSS (Residual Sum of Squares) is calculated like so:

$$
RSS = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2
$$

In [ ]:
rss = sum((y - predictions) ** 2)
rss

In [ ]:
rse = sqrt(rss / (len(X) - 2))
rse

The RSE is considered a measure of the *lack of fit* of the model to the data. So in our case, how well does our model fit the data?
If our model is very good, then the model predictions will be very close to the true outcome values, in other words $\hat{y_i} \approx y_i$ then the RSE will be small. If the RSE is high, then that means that our predictions are not good, and this indicates that the model doesn't fit the data very well.

Now as you can imagine the RSE does not have an upper limit. This makes it a bit difficult to interpret, because the values can get very large. That is why we use the $R^2$ value, which is a standardized measure of fit.

The $R^2$ value is the proportion of variance explained by the model. It is calculated as:

$$
R^2 = 1 - \frac{RSS}{TSS}
$$

Where TSS is the Total Sum of Squares:

$$
TSS = \sum_{i=1}^{n}(y_i - \bar{y})^2
$$

The "expanded" form of the $R^2$ value is (writing it like this may make it a bit easier to remember or understand what's going on):

$$
R^2 = 1 - \frac{\sum_{i=1}^n\left( y_i - \hat{y_i} \right)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}
$$

In [ ]:
r2 = 1 - rss / sum((y - y.mean()) ** 2)
r2

We can also get this value from the `statsmodels` summary from the table (top right corner), or programmatically:

In [ ]:
model.rsquared

iii) Is the relationship between the predictor and the response positive or negative?

The relationship between the predictor and the response is negative. This is because the coefficient for `horsepower` is negative. This means that if `horsepower` increases, then `mpg` decreases. This is a negative relationship.

iv) What is the predicted `mpg` associated with a `horsepower` of 98? What are the associated 95% confidence and prediction intervals?

The predicted `mpg` associated with a `horsepower` of 98 is calculated as:

$$
\hat{y} = \hat{\beta_0} + \hat{\beta_1} \times x
$$
    
Where $x = 98$.

In [ ]:
task1_iv = beta0 + beta1 * 98
task1_iv

To get the confidence and prediction intervals:

In [ ]:
se_pred_ci = residual_std_error * np.sqrt((1/len(X)) + ((98 - mean_x) ** 2 / sum((X['horsepower'] - mean_x) ** 2)))
se_pred_ci

In [ ]:
ci_task1_iv_lower = task1_iv - t_value * se_pred_ci
ci_task1_iv_upper = task1_iv + t_value * se_pred_ci

ci_task1_iv_lower, ci_task1_iv_upper

In [ ]:
se_pred_pi = residual_std_error * np.sqrt(1 + (1/len(X)) + ((98 - mean_x) ** 2 / sum((X['horsepower'] - mean_x) ** 2)))
pi_task1_iv_lower = task1_iv - t_value * se_pred_pi
pi_task1_iv_upper = task1_iv + t_value * se_pred_pi

pi_task1_iv_lower, pi_task1_iv_upper

Or, taking them from `statsmodels` directly:

In [ ]:
model.get_prediction([1, 98]).summary_frame(alpha=0.05)

b) Plot the response and the predictor. Display the least squares regression line.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=auto_df['horsepower'], y=auto_df['mpg'], mode='markers', name='Data'))

predictions = model.predict(X)
fig.add_trace(go.Scatter(x=X['horsepower'], y=predictions, mode='markers', name='Predictions'))

# get regression line from model
beta0 = model.params['const']
beta1 = model.params['horsepower']

x_line = np.linspace(auto_df['horsepower'].min(), auto_df['horsepower'].max(), 100)
y_line = beta0 + beta1 * x_line
fig.add_trace(go.Scatter(x=x_line, y=y_line, mode='lines', name='Regression Line'))

fig.update_layout(
    title="Regression Plot of Horsepower vs MPG",
    xaxis_title="Horsepower",
    yaxis_title="MPG"
)

fig.show()

c) Perform a multiple linear regression on the dataset. Use `mpg` as the response variable and both `horsepower` and `weight` as the predictor variables. Use `statsmodels` to solve this task (instead of doing it manually). Does this model perform better than the simple linear regression?

In [ ]:
X = auto_df[['horsepower', 'weight']]
X = sm.add_constant(X)
y = auto_df['mpg']
model = sm.OLS(y, X).fit()
model.summary()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(auto_df, x='horsepower', y='weight', z='mpg', opacity=0.5)
fig.update_traces(marker=dict(size=5))
fig.update_layout(scene=dict(
    xaxis_title='Horsepower',
    yaxis_title='Weight',
    zaxis_title='MPG'
))

x_surf, y_surf = np.meshgrid(np.linspace(auto_df['horsepower'].min(), auto_df['horsepower'].max(), 100),
                             np.linspace(auto_df['weight'].min(), auto_df['weight'].max(), 100))

exog = pd.DataFrame({'const': np.ones(10000), 'horsepower': x_surf.ravel(), 'weight': y_surf.ravel()})
out = model.predict(exog=exog)

fig.add_trace(go.Surface(x=x_surf, y=y_surf, z=out.values.reshape(x_surf.shape), showscale=False))

fig.update_layout(height=800)
fig.show()

# Task 2 - Pokemon Linear Regression

- a) Perform a simple linear regression on the `pokemon.csv` dataset. Use `generation` as the response variable and `speed` as the predictor variable. Use `statsmodels` to solve this task.
    - i) Is there a relationship between the predictor and the response?
    - ii) How strong is the relationship between the predictor and the response?
    - iii) Is the relationship between the predictor and the response positive or negative?
- b) 💬 Discuss in pairs: How can you improve the model or build a new, better model?

In [ ]:
pokemon_df = pd.read_csv('pokemon.csv')
pokemon_df

In [ ]:
x = pokemon_df[['speed']]
X = sm.add_constant(x)
y = pokemon_df['generation']
model = sm.OLS(y, X).fit()
model.summary()

We see here that our p-value for the coefficient of `speed` is $0.574$. This means that we cannot reject the null hypothesis that there is no relationship between `speed` and `generation`. This means that we cannot say that there is a relationship between `speed` and `generation`.

We can also see from the table that the $R^2$ is 0, which means that the model does not explain any of the variance in `generation`. This is another indicator that there is no relationship between `speed` and `generation`.

b) How can you improve the model or build a new, better model?

One way to improve the model is to use a different predictor variable. We can see from the table that the coefficient for `speed` is not significant. This means that `speed` is not a good predictor for `generation`. Let's see which variables are the best predictors for `generation`. We can do this by calculating the correlation between all variables and `generation`.

In [ ]:
pokemon_df[['attack', 'defense', 'sp_attack', 'sp_defense', 'hp', 'generation', 'speed']].corr()['generation']

From this analysis, it seems like `attack` and `hp` may be useful, due to their (comparatively) high correlation. Our original predictor, `speed`, has a slightly negative correlation with `generation`, which is why it is not a good predictor.

In [ ]:
x = pokemon_df[['attack', 'hp']]
X = sm.add_constant(x)
y = pokemon_df['generation']
model = sm.OLS(y, X).fit()
model.summary()

`hp` fails to be significant, but `attack` is significant. This means that `attack` is a good predictor for `generation`. Let's try a model with *only* this predictor.

In [ ]:
x = pokemon_df['attack']
X = sm.add_constant(x)
y = pokemon_df['generation']
model = sm.OLS(y, X).fit()
model.summary()

For the single-predictor model the $R^2$ decreased, compared to the one with two predictors. This may seem confusing, because the predictor we removed was not statistically significant (it failed the p-test). This is a problem with the $R^2$ metrics - it does not care whether the predictors are statistically significant. That's why the adjusted $R^2$ is often used. The adjusted $R^2$ is calculated as:

$$
R^2_{adj} = 1 - \left( \frac{\left(1 - R^2\right) \times \left(n - 1\right)}{n - k - 1} \right)
$$

Where $n$ is the number of observations and $k$ is the number of predictors. This metric penalizes the $R^2$ value for having too many predictors. We can see that in our case for both models this is (ignoring rounding) the same.

# Task 3 - Databases

In this task we have access to the Spotify database (not really, but let's pretend it's the actual Spotify database - it's a dataset taken from [Kaggle](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)).

The focus of this course is not on databases, but they are something you will encounter in Data Science projects. If you want to learn more about databases, there is a course called *Advanced Databases* that you can take!

We will pretend like we know nothing about the database and will try to explore it.


- a) Load the database into a DataFrame. You can use [`pd.read_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) to do this.
- b) 💬 Discuss in pairs: Do you notice any issues? If so, what are they?
    - There are multiple things that could be improved in this DataFrame. Depending on what step(s) you would do next with this data you can ignore them, or you have to find ways to fix or work around them.
    - In this scenario, we want to train a model that can detect if a text is explicit or not - we want to train a model on this data using the `explicit` and `track_name` columns. There is one main issue that we need to fix before we can do this.
    - *Hint*: There is a cell provided that highlights the issue. Try to find it without using it first - in a real project checking for potential issues in your data is a step that you need to do without any help.
- c) Fix the issue(s) you have found.


In [ ]:
# the cell here may not display anything, depending on how you run the notebook - if it doesn't work you can ignore it
# it should work if you open your jupyter notebook in a browser
from IPython.display import display, HTML

spotify_embed_code = """
<iframe style="border-radius:12px" src="https://open.spotify.com/embed/track/3QeOoGlEnOfsgghQvsLenS?utm_source=generator&theme=0" width="100%" height="352" frameBorder="0" allowfullscreen="" allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture" loading="lazy"></iframe>
"""
display(HTML(spotify_embed_code))

In our case the database is a very simple `sqlite` database. It's just one file - `database.db`. Normally databases are much more complex and consist of multiple files and sharing them is not as trivial as it is in our case.

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

In [ ]:
# get all column names
cursor.execute("PRAGMA table_info(spotify)")
columns = cursor.fetchall()
columns

Solution a):

In [ ]:
spotify_df = pd.read_sql_query("SELECT * FROM spotify", conn)
spotify_df

Task b) hint cell:

In [ ]:
spotify_df[spotify_df['artists'] == 'Nightwish']

Solution b):


There are odd characters in the `track_name` column. Something went wrong. It looks like the data was saved incorrectly in the database.
This can happen sometimes - it's an issue with encoding. Usually what we want is UTF-8 encoding. So for now let's try out various wrong encodings, and see which one works in our case, to find a way to fix the broken data.

Solution c):

In [ ]:
def fix_encoding(broken_text, wrong_encoding, correct_encoding='utf-8'):
    try:
        return broken_text.encode(wrong_encoding).decode(correct_encoding)
    except UnicodeDecodeError:
        return broken_text
    except AttributeError:
        return broken_text

In [ ]:
import encodings
import pkgutil

available_encodings = set([modname for _, modname, _ in pkgutil.iter_modules(encodings.__path__)])
available_encodings

In [ ]:
broken_text = "Ãlan"

for enc in available_encodings:
    try:
        fixed_text = fix_encoding(broken_text, enc)
        print(f"Using encoding {enc}: {fixed_text}")
    except Exception as e:
        # print(f"Error using encoding {enc}: {e}")
        pass

Of these options, a bunch of them look like they could work. Let's try `iso8859_10` on the entire column.

In [ ]:
# spotify_df['track_name'].apply(fix_encoding, wrong_encoding='iso8859_10')

This didn't work - let's try `latin_1`.

In [ ]:
spotify_df['track_name'].apply(fix_encoding, wrong_encoding='latin_1')

This worked! Now we can actually apply it to the DataFrame.

In [ ]:
spotify_df['track_name'] = spotify_df['track_name'].apply(fix_encoding, wrong_encoding='latin_1')
spotify_df